# Advanced OOP

In [1]:
# Notebook Set-Up Commands

def print_error(error):
    """ Print Error
    
    Function to print exceptions in red.
    
    Parameters
    ----------
    error : string
        Error message
    
    """
    print('\033[1;31m{}\033[1;m'.format(error))

## 2 Inheritance

---

A powerful property of classes is the ability to inherit attributes and methods from other classes. This helps avoid repetition of code, which makes it easier to develop and maintain. 

<br>

### Inheriting Attributes

We will begin by defining a simple class with a single static method that will act as a *parent* class.

In [2]:
# Define a parent class
class Parent:
    
    @staticmethod
    def add(a, b):
        return a + b

# Print the parent dictionary
print('Parent.__dict__ =', Parent.__dict__)

Parent.__dict__ = {'__module__': '__main__', 'add': <staticmethod object at 0x7f2bbc5b7940>, '__dict__': <attribute '__dict__' of 'Parent' objects>, '__weakref__': <attribute '__weakref__' of 'Parent' objects>, '__doc__': None}


Note that this class has no special properties, it the same as classes we saw in the previous notebook.

Now we will define a second class, also with a single static function. This class will serve as a *child* class that will inherit from the parent class. This is done by simply passing the parent class name in `()` when defining the child class. 

In [3]:
# Define a child class
class Child(Parent):
    
    @staticmethod
    def subtract(a, b):
        return a - b

print('Child.__dict__ =', Child.__dict__)

Child.__dict__ = {'__module__': '__main__', 'subtract': <staticmethod object at 0x7f2bbc59d520>, '__doc__': None}


We can see that the contents of the child class dictionary are only what is expected from a normal class definition. Using the special `__bases__` attribute, however, we can see a tuple of classes that the child inherits from.

In [4]:
print('Child.__bases__ =', Child.__bases__)

Child.__bases__ = (<class '__main__.Parent'>,)


Finally, we can demonstrate that the child has indeed inherited new attributes from the parent as follows.

In [5]:
print('1 + 2 =', Child.add(1, 2))
print('3 - 2 =', Child.subtract(3, 2))

1 + 2 = 3
3 - 2 = 1


We can clearly see that `Child` has inhertited the static `add` method from `Parent`.

In the following example, we can see that this works for any type of class attribute.

In [6]:
class Parent:
    
    x = 1
    
class Child(Parent):
    
    @classmethod
    def show(cls):
        return print('x =', cls.x)

Child.show()

x = 1


### Overriding

We saw in the previous notebook that class instance attributes will override class attributes of the same name. The same happens with child attributes with respect to parent attributes. 

In [7]:
class Parent:
    
    x = 1
    
class Child(Parent):
    
    x = 2
    
    @classmethod
    def show(cls):
        return print('x =', cls.x)

Child.show()

x = 2


### Hierachy

It is possible to define a hierarchy of parent classes, the attributes of which will be inheried by a given child class.

In [36]:
class GrandParent:
    
    x = 1
    name = 'Grandparent'

class Parent(GrandParent):
    
    y = 2
    name = 'Parent'
    
class Child(Parent):
    
    z = 3
    name = 'Child'

print(Child.name, Child.x, Child.y, Child.z)
print(GrandParent.name)

Child 1 2 3
Grandparent


Note that overriding will also act hierachically, meaning that the last class in the chain will be given precedence.

<br>

### Handling Instantiation 

If the parent class contains an `__init__` method, this too can be inherited by the child.

In [9]:
class Parent:
    
    def __init__(self, value):
        self.myattr = value
    
class Child(Parent):
    
    def show(self):
        return print('myattr =', self.myattr)
    
Child('A string').show()

myattr = A string


In fact it also works the other way around.

In [10]:
class Parent:
    
    def show(self):
        return print('myattr =', self.myattr)
    
class Child(Parent):
    
    def __init__(self, value):
        self.myattr = value
    
Child('A string').show()

myattr = A string


This is because the child will lookup any attributes not listed in its own dictionary in all the parent (or base) classes.

If both classes have an `__init__` method only the child class will be initialised (see [Overriding](#Overriding)).

In [11]:
class Parent:
    
    def __init__(self):
        self.pval = 'parent value'
    
class Child(Parent):
    
    def __init__(self):
        self.cval = 'child value'
    
inst = Child()
print('cval =', inst.cval)

cval = child value


In [12]:
try:
    print('pval =', inst.pval)
except Exception as error:
    print_error(error)

'Child' object has no attribute 'pval'


We can fix this by manually instantiating the parent class.

In [13]:
class Parent:
    
    def __init__(self):
        self.pval = 'parent value'
    
class Child(Parent):
    
    def __init__(self):
        self.cval = 'child value'
        Parent.__init__(self)
    
inst = Child()
print('cval =', inst.cval)
print('pval =', inst.pval)

cval = child value
pval = parent value


Python has a useful shortcut to make this process easier using the [`super`](https://docs.python.org/3/library/functions.html?highlight=super#super)function. This allows us to instantiate the parent without naming it, this particularly useful if the name of the parent class were to change.

In [14]:
class Parent:
    
    def __init__(self):
        self.pval = 'parent value'
    
class Child(Parent):
    
    def __init__(self):
        self.cval = 'child value'
        super().__init__()
    
inst = Child()
print('cval =', inst.cval)
print('pval =', inst.pval)

cval = child value
pval = parent value


### Multiple Parents

It is possible for a child to inherit attributes from multiple parents.

In [15]:
class Mother:
    
    def __init__(self, value):
        self.mval = value
        
    def show_mother(self):
        print('mother value =', self.mval)
        
class Father:
    
    def __init__(self, value):
        self.fval = value
        
    def show_father(self):
        print('father value =', self.fval)
    
class Child(Mother, Father):
    
    def __init__(self, value1, value2, value3):
        self.cval = value3
        Mother.__init__(self, value1)
        Father.__init__(self, value2)
        
    def show_child(self):
        print('child value =', self.cval)
        
inst = Child(1, 2, 3)
inst.show_mother()
inst.show_father()
inst.show_child()

mother value = 1
father value = 2
child value = 3


Note that the parent classes were explicitly instantiated in the previous example. Attempting the same thing using the `super` function will raise an error.

In [37]:
class Mother:
    
    def __init__(self, value):
        self.mval = value
        
    def show_mother(self):
        print('mother value =', self.mval)
        
class Father:
    
    def __init__(self, value):
        self.fval = value
        
    def show_father(self):
        print('father value =', self.fval)
    
class Child(Mother, Father):
    
    def __init__(self, value1, value2):
        self.cval = value2
        super().__init__(value1)
        
    def show_child(self):
        print('child value =', self.cval)
        
inst = Child(1, 2)

In [38]:
try:
    inst.show_mother()
    inst.show_father()
    inst.show_child()
except Exception as error:
    print_error(error)

mother value = 1
'Child' object has no attribute 'fval'


As we can see the `Mother` parent class was properly instantiated, but the `Father` parent class was not. The reasons for this are explained in the following subsection.

> In general, caution should be used when building a class architecture that requires inheritance for multiple parent classes.

<br>

### Method Resolution Order

The Method Resolution Order (MRO) dictates the order in which the child class will search the base classes for a given attribute. We can see the MRO of the child class from the previous example using the `__mro__` attribute as follows.

In [18]:
print('Child.__mro__ =', Child.__mro__)

Child.__mro__ = (<class '__main__.Child'>, <class '__main__.Mother'>, <class '__main__.Father'>, <class 'object'>)


We can see that the `Mother` class appears before the `Father` class in the MRO, hence the `super` method in `Child` instantiates `Mother`. 

## 3 Composition
---

Inhertitance is not the only way in which a class can access attributes from other classes. This comes back one of main take-away messages of this tutorial, namely that everything in Python is an object and any object can be assigned as an attribute.

This means that a class attribute can actually be an instance of another class.

In [42]:
class Composer:
    
    def __init__(self):
        self.myattr = 'composer value'
    
class myClass:
    
    def __init__(self):
        self.comp = Composer()

inst = myClass()
print("myattr =", inst.comp.myattr)

myattr = composer value


This can particularly useful as we can pass classes or class intances to our new class without knowing anything about the other classes. 

In [43]:
class Star:
    
    def __init__(self):
        self.whoami = 'I am a star!'
        
class Galaxy:
    
    def __init__(self):
        self.whoami = 'I am a Galaxy!'
    
class myClass:
    
    def __init__(self, composer):
        self.comp = composer()

inst1 = myClass(Star)
inst2 = myClass(Galaxy)
print("whoami =", inst1.comp.whoami)
print("whoami =", inst2.comp.whoami)

whoami = I am a star!
whoami = I am a Galaxy!


We can even achieve inheritance-like properties if we know the name of a given attribute that the composer class should have.

In [22]:
class Star:
    
    def __init__(self):
        self.whoami = 'I am a star!'
        
class Galaxy:
    
    def __init__(self):
        self.whoami = 'I am a Galaxy!'
    
class myClass:
    
    def __init__(self, composer):
        self.whoami = composer().whoami

inst1 = myClass(Star)
inst2 = myClass(Galaxy)
print("whoami =", inst1.whoami)
print("whoami =", inst2.whoami)

whoami = I am a star!
whoami = I am a Galaxy!


> Note: This will break if the composer does not have an attribute called `whoami`.

Composition also alows preinitialised class instances to be passed.

In [23]:
class Star:
    
    def __init__(self, mag):
        self.mag = mag
        
class Galaxy:
    
    def __init__(self, mag):
        self.mag = mag
    
class myClass:
    
    def __init__(self, composer):
        self.comp = composer
        self.mag = self.comp.mag

        
star = Star(11.05)
inst1 = myClass(star)
inst2 = myClass(Galaxy(8.2))
print("mag =", inst1.mag)
print("mag =", inst2.mag)

mag = 11.05
mag = 8.2


This can be useful, especially if we want to be able to change the original instance attributes.

In [24]:
inst1.mag = 12.3
inst1.comp.mag = 13.1
print("inst1 mag =", inst1.mag)
print("star mag =", star.mag)

inst1 mag = 12.3
star mag = 13.1


Notice how the `mag` attribute of the instance `star` changed!

## 4 Abstract Classes
---

Abstract classes are classes that contain abstract methods. So it makes sense to move right along and explain what an abstract method is.

<br>

### Abstract Methods

An abstract method is a method that is defined but not implemented. This is useful for defining parent classes that impose some conditions on the child.

In [25]:
from abc import ABC, abstractmethod

class Parent(ABC):
    
    @abstractmethod
    def get_id(self):
        pass

print ('Parent.__dict__', Parent.__dict__)

Parent.__dict__ {'__module__': '__main__', 'get_id': <function Parent.get_id at 0x7f2bbc548790>, '__dict__': <attribute '__dict__' of 'Parent' objects>, '__weakref__': <attribute '__weakref__' of 'Parent' objects>, '__doc__': None, '__abstractmethods__': frozenset({'get_id'}), '_abc_impl': <_abc._abc_data object at 0x7f2bbc560380>}


We can see that the method `get_id` has been defined but does nothing.

We now define a child class that inherits this parent.

In [26]:
class Child(Parent):
    
    def __init__(self):
        self.whoami = 'I am a child!'

In [27]:
try:
    inst = Child()
except Exception as error:
    print_error(error)

Can't instantiate abstract class Child with abstract method get_id


This raises an error when trying to create an instance. This is because the child class needs to override the abstract method `get_id`. 

In [44]:
class Child(Parent):
    
    def __init__(self):
        self.get_id()
        
    def get_id(self):
        self.whoami = 'I am a child!'
        
inst = Child()
print(inst.whoami)

I am a child!


This allows the parent class, which may be written with no knowledge of the properties of the child class, to impose some conditions on what a given child class should do.

<br>

### Abstract Properties

In the previous notebook we learned how class properties can be defined to impose conditions on the values of certain class attributes. Using Abstract methods we can combine these tools to ensure that a given class has the properties we want.

We start by defining an abstract class with the abstract property `whoami` and its corresponding setter.

In [29]:
class AstroObject(ABC):
    
    @property
    @abstractmethod
    def whoami(self):
        pass
    
    @whoami.setter
    @abstractmethod
    def whoami(self, value):
        pass

We can then define a child class that inherits this abstract class.

In [30]:
class Star(AstroObject):
    
    def __init__(self):
        self.whoami = 'I am a star!'

In [31]:
try:
    star = Star()
except Exception as error:
    print_error(error)

Can't instantiate abstract class Star with abstract method whoami


We can see that while a value for the attribute is provided the property is not explicitly defined. This way we can impose that the `Star` class must define a getter and setter for the attribute `whoami`.

In [32]:
class Star(AstroObject):
    
    def __init__(self):
        self.whoami = 'I am a star!'
    
    @property
    def whoami(self):
        return self._whoami
    
    @whoami.setter
    def whoami(self, value):
        if not isinstance(value, str):
            raise ValueError('whoami must be a string')
        self._whoami = value

star = Star()
print("whoami =", star.whoami)


whoami = I am a star!


This in turn could be used in conjunction with what we learned about class composition to ensure that composers have the properties needed to work with the child class.

## 5 Exercises
---

1. Create a parent class and a child class that can identify its progenitor. 
    1. Your parent class should have the class attribute `parent_name` with the value of your choice.
    1. Your child class should have the attribute `name` with the value of your choice.
    1. Printing an instance of your child class should contain its name and its parent's name. *e.g.* 
    
    ```python
    print(Child('Thor'))
    Thor Odinson
    ```

In [33]:
# Add your solution here

2. Define a class that can be initialised with composer classes that have been constrained by an abstract class.
    1. Define an abstract class called `EarthAttr` that has the abstract method `whatami`, whcih should return a string of your choice.
    1. Define at least two composer classes (*e.g.* `Moon` and `Core`) that satisfy the requirements of `EarthAttr`.
    1. Define a class called `Earth` that composes these classes to get the `whatami` attribute.
    1. Printing an instance of your `Earth` class should include the value of `whatami`. *e.g.*
    
    ```python
    print(Earth(Moon))
    The Earth has a moon!
    ```
    
    5. Finally, define a final composer class (*e.g.* `Lake`) and demonstrate that it will not instantiate if not correctly constrained by `EarthAttr`. You should get the following error:
    
    ```bash
        'Cant instantiate abstract class Lake with abstract methods whatami'
    ```

3. Create a deck of cards class (taken from https://www.rithmschool.com/courses/python-fundamentals-part-2/python-object-oriented-programming-exercises). Internally, the deck of cards should use another class, a card class. Your requirements are:

The Deck class should have a deal method to deal a single card from the deck
After a card is dealt, it is removed from the deck.
There should be a shuffle method which makes sure the deck of cards has all 52 cards and then rearranges them randomly.
The Card class should have a suit (Hearts, Diamonds, Clubs, Spades) and a value (A,2,3,4,5,6,7,8,9,10,J,Q,K).

Use shuffle from random to shuffle a list of objects.

In [45]:
from random import shuffle